In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import math
import torch
import pickle
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
import joblib

# check xgboost version
from xgboost import XGBRegressor

#torch stuff
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

#from torch_geometric.utils import dense_to_sparse


import lightgbm as lgb
from lightgbm import LGBMRegressor

import holidays
us_holidays = holidays.country_holidays('US')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

directory = '/content/drive/MyDrive/Thesis'
data_dir = directory + "/Data"
models_dir = directory + "/models"

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
from hyperopt.pyll.base import scope
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [4]:
df = pd.read_csv(f'{data_dir}/final_model_input_partial_scale_3.csv')

df['started_at_hourly'] = pd.to_datetime(df['started_at_hourly'])
df = df.sort_values(by=['start_station_cluster', 'started_at_hourly'])

df_test = df[pd.to_datetime(
    df['started_at_hourly']) >= pd.to_datetime("2024-01-01 00:00:00")]
df_train = df[pd.to_datetime(
    df['started_at_hourly']) < pd.to_datetime("2024-01-01 00:00:00")]

del df

In [5]:
target_scaler = joblib.load(f'{models_dir}/target_scaler.sav')

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.5.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
execfile('/content/drive/MyDrive/Thesis/models_training/model_training.py')

Using device: cpu


# GNN Variance Embedding

In [7]:
FEATURES = ['start_station_cluster', 'started_at_year',
            'started_at_month', 'started_at_day',
            'started_at_hour',
            'started_at_week',
            'started_at_quarter',
            'started_at_dayofweek',
            'is_holiday',
            #'flag_added',
            'temp',
            'dwpt',
            'rhum',
            'prcp',
            'wdir',
            'wspd',
            'pres',
            'coco',
            "demand_lag_1_h",
            "demand_lag_2_h",
            "demand_lag_24_h",
            "temp_lag_1_h",
            "temp_lag_2_h",
            "temp_lag_24_h",
            "prcp_lag_1_h",
            "prcp_lag_2_h",
            "prcp_lag_24_h",
            "rhum_lag_1_h",
            "rhum_lag_2_h",
            "rhum_lag_24_h",
            "wspd_lag_1_h",
            "wspd_lag_2_h",
            "wspd_lag_24_h",
            "total_demand_1h",
            "demand_degrees_1h",
]
FEATURES.extend([f"dim_var_{i}" for i in range(50)])

## XGBoost

In [8]:


model, model_preds = train_model(df_train, df_test, FEATURES, model='xgb')

overall
MSE: 3.6794573989792814
RMSE: 1.9181911789441848
MAE: 1.0264590525877988
MAPE: 0.4178085847938585

Non-zero
MSE: 7.460762608077842
RMSE: 2.7314396585093808
MAE: 1.7112004481930954
MAPE: 0.3581432178725427

Zeros
MSE: 0.5877428952341267
RMSE: 0.7666439168441413
MAE: 0.46659287225048623
MAPE: 0.46659287225048623


In [ ]:
#11_01
3.1785156823909997
1.7828392194449278
0.374173850198251

#11-09 scaled
3.6949065265972303
1.9222139648325391
0.4156726768881891

#11-09 unscaled
3.6949065102953313
1.9222139605921427
0.4156740721031502

#11-28
3.6794573989792814
1.9181911789441848
0.4178085847938585

0.4178085847938585

In [ ]:
suf = "_f_xgb_11_28"


model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model

## LightGBM

In [9]:
model, model_preds = train_model(df_train, df_test, FEATURES, model='lgbm')

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 2.401502 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15689
[LightGBM] [Info] Number of data points in the train set: 1140300, number of used features: 84
[LightGBM] [Info] Start training from score 1.005143
overall
MSE: 3.8116125514504833
RMSE: 1.9523351534637907
MAE: 1.057537647838013
MAPE: 0.43218367768957955

Non-zero
MSE: 7.710290968245343
RMSE: 2.7767410697156016
MAE: 1.7563574229339898
MAPE: 0.36616755930693445

Zeros
MSE: 0.6239300079627109
RMSE: 0.7898924027756634
MAE: 0.48616054001570674
MAPE: 0.48616054001570674


In [ ]:
#11-28
3.8116125514504833
1.9523351534637907
0.43218367768957955

In [ ]:
suf = "_f_lgbm_11_28"


model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model

## Random Forest

In [10]:
model, model_preds = train_model(df_train, df_test, FEATURES, model='rf')

overall
MSE: 4.176526598412972
RMSE: 2.043655205364391
MAE: 1.0894263851495312
MAPE: 0.45485283627293793

Non-zero
MSE: 8.34754281652863
RMSE: 2.8892114523739223
MAE: 1.788550626481544
MAPE: 0.3778652267155616

Zeros
MSE: 0.766172071654559
RMSE: 0.875312556550264
MAE: 0.5178003361215292
MAPE: 0.5178003361215292


In [ ]:
#11_01
3.6042873699256828
1.8984960810930538
0.4084309933647605

#11-28
4.152445644395686
2.037755050146039
0.4570231926836906

0.4084309933647605

In [ ]:
suf = "_f_rf_11_28"


model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model